In [21]:
from sklearn import tree #decision tree
from sklearn.ensemble import RandomForestRegressor #random Forest
from sklearn.model_selection import cross_val_score # evaluate our model
from sklearn.model_selection import GridSearchCV # gridsearch to find best hyper parameters
import pandas as pd
import numpy as np
import datetime as dt

In [22]:
Schedule_and_stats_played = pd.read_csv("2020_2021_Played_Games.csv", index_col=0)
Schedule_and_stats_unplayed = pd.read_csv("2020_2021_Unplayed_Games.csv", index_col=0)

In [23]:
Schedule_and_stats_played["Date"] = pd.to_datetime(Schedule_and_stats_played["Date"])
Schedule_and_stats_unplayed["Date"] = pd.to_datetime(Schedule_and_stats_unplayed["Date"])


In [24]:
Schedule_and_stats_played[(Schedule_and_stats_played['home_neutral']=='Michigan')]

,Date,visitor_neutral,PTS,home_neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
893,2020-11-25,Bowling Green State,82,Michigan,96,Bowling Green State,21,12,9,0.571,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
894,2020-12-02,Ball State,65,Michigan,84,Ball State,18,7,11,0.389,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
895,2020-11-29,Oakland,71,Michigan,81,Oakland,27,10,17,0.370,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
896,2020-12-09,Toledo,71,Michigan,91,Toledo,24,18,6,0.750,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
897,2021-01-12,Wisconsin,54,Michigan,77,Wisconsin,24,16,8,0.667,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
898,2020-12-06,UCF,58,Michigan,80,UCF,19,8,11,0.421,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
899,2020-12-13,Penn State,58,Michigan,62,Penn State,19,7,12,0.368,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
900,2021-01-19,Maryland,63,Michigan,87,Maryland,24,14,10,0.583,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
901,2021-02-18,Rutgers,64,Michigan,71,Rutgers,21,12,9,0.571,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
902,2021-01-06,Minnesota,57,Michigan,82,Minnesota,23,13,10,0.565,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0


In [25]:
Schedule_and_stats_unplayed[(Schedule_and_stats_unplayed['visitor_neutral']=='Michigan') | (Schedule_and_stats_unplayed['home_neutral']=='Michigan')]

,Date,visitor_neutral,PTS,home_neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
60,2021-03-02,Illinois,0,Michigan,0,Illinois,21,16,5,0.762,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
61,2021-02-25,Iowa,0,Michigan,0,Iowa,23,17,6,0.739,...,227.0,296.0,0.767,158.0,661.0,285.0,73.0,85.0,205.0,252.0
253,2021-02-27,Michigan,0,Indiana,0,Michigan,17,16,1,0.941,...,353.0,529.0,0.667,198.0,773.0,317.0,126.0,88.0,269.0,400.0


In [26]:
X = Schedule_and_stats_played.drop(["Date","visitor_neutral","home_neutral","School_x","School_y","PTS","PTS.1"], axis=1)
Y = Schedule_and_stats_played[["PTS","PTS.1"]]

In [27]:
reg = tree.DecisionTreeRegressor(random_state=0)
reg = reg.fit(X, Y)

In [28]:
### cross validations
print(cross_val_score(reg, X, Y, cv=5))

#First model performs HORRIBLY!

[-0.57895122 -0.43695159 -0.61352867 -0.59698713 -0.68685031]


In [29]:
# use grid search to determine decision tree depth
params = {"max_depth":range(1,10)}
treereg = tree.DecisionTreeRegressor(random_state=0)
reg2 = GridSearchCV(estimator = treereg, param_grid=params)
reg2.fit(X,Y)
reg2.best_params_


{'max_depth': 4}

In [30]:
means = reg2.cv_results_['mean_test_score']
stds = reg2.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, reg2.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()


0.027 (+/-0.034) for {'max_depth': 1}

0.071 (+/-0.050) for {'max_depth': 2}

0.082 (+/-0.044) for {'max_depth': 3}

0.083 (+/-0.034) for {'max_depth': 4}

0.048 (+/-0.021) for {'max_depth': 5}

0.020 (+/-0.034) for {'max_depth': 6}

-0.073 (+/-0.100) for {'max_depth': 7}

-0.174 (+/-0.099) for {'max_depth': 8}

-0.248 (+/-0.065) for {'max_depth': 9}



In [31]:
reg3 = RandomForestRegressor(random_state=0)
reg3.fit(X,Y)
print(cross_val_score(reg3, X, Y, cv=5))

[0.24614074 0.22857139 0.18077521 0.19676021 0.17456752]


In [35]:
#get game to predict.  Gonzaga vs Northwestern state
month, day, year = 2, 23, 2021
date = pd.to_datetime(str(year)+"-"+str(month)+"-"+str(day))
games = Schedule_and_stats_unplayed[Schedule_and_stats_unplayed["Date"]==date]
teams = games[["visitor_neutral", "home_neutral"]]
games = games.drop(["Date","visitor_neutral","home_neutral","School_x","School_y","PTS","PTS.1"], axis=1)
games

,G_x,W_x,L_x,W-L%_x,SRS_x,SOS_x,Conf W_x,Conf L_x,Home W_x,Home L_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
31,18,10,8,0.556,-11.67,-7.31,6,6,7,4,...,221.0,292.0,0.757,160.0,742.0,347.0,108.0,52.0,213.0,325.0
35,14,7,7,0.500,1.60,4.98,4,6,5,2,...,115.0,177.0,0.650,81.0,371.0,143.0,78.0,39.0,148.0,205.0
54,18,2,16,0.111,-1.05,10.39,0,13,2,8,...,205.0,291.0,0.704,215.0,637.0,309.0,165.0,69.0,210.0,286.0
56,21,16,5,0.762,24.77,11.58,12,3,10,2,...,258.0,349.0,0.739,226.0,767.0,338.0,88.0,89.0,274.0,393.0
62,19,14,5,0.737,16.36,8.15,9,4,10,1,...,242.0,368.0,0.658,243.0,785.0,316.0,126.0,40.0,360.0,413.0
63,23,14,9,0.609,9.94,7.03,8,8,9,3,...,239.0,303.0,0.789,167.0,574.0,268.0,88.0,26.0,153.0,263.0
64,24,17,7,0.708,16.37,9.89,11,5,11,1,...,253.0,370.0,0.684,218.0,750.0,249.0,125.0,75.0,259.0,390.0
65,19,11,8,0.579,13.16,8.43,7,6,9,3,...,246.0,347.0,0.709,172.0,636.0,254.0,83.0,68.0,209.0,290.0
67,18,14,4,0.778,4.56,-2.37,12,3,9,0,...,213.0,308.0,0.692,163.0,578.0,300.0,118.0,42.0,204.0,278.0
72,21,12,9,0.571,-5.85,-5.50,6,6,9,4,...,203.0,308.0,0.659,182.0,596.0,265.0,121.0,69.0,192.0,277.0


In [36]:
with pd.option_context('display.max_columns', None):
  display(Schedule_and_stats_unplayed.head(2))

,Date,visitor_neutral,PTS,home_neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,SRS_x,SOS_x,Conf W_x,Conf L_x,Home W_x,Home L_x,Away W_x,Away L_x,Tm._x,Opp._x,MP_x,FG_x,FGA_x,FG%_x,3P_x,3PA_x,3P%_x,FT_x,FTA_x,FT%_x,ORB_x,TRB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,School_y,G_y,W_y,L_y,W-L%_y,SRS_y,SOS_y,Conf W_y,Conf L_y,Home W_y,Home L_y,Away W_y,Away L_y,Tm._y,Opp._y,MP_y,FG_y,FGA_y,FG%_y,3P_y,3PA_y,3P%_y,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
0,2021-02-22,Texas Tech,0,Oklahoma State,0,Texas Tech,21,14,7,0.667,16.74,6.03,6,6,8,4,5,2,1546.0,1321.0,845.0,527.0,1210.0,0.436,133.0,392.0,0.339,359.0,498.0,0.721,252.0,750.0,265.0,146.0,87.0,229.0,363.0,Oklahoma State,20,14,6,0.7,12.27,7.12,7,6,8,3,6,3,1515.0,1412.0,815.0,543.0,1196.0,0.454,133.0,391.0,0.34,296.0,423.0,0.7,227.0,788.0,269.0,151.0,92.0,311.0,340.0
1,2021-03-01,Oklahoma,0,Oklahoma State,0,Oklahoma,19,14,5,0.737,16.36,8.15,9,4,10,1,4,4,1441.0,1285.0,770.0,516.0,1167.0,0.442,151.0,454.0,0.333,258.0,345.0,0.748,201.0,719.0,259.0,138.0,63.0,202.0,275.0,Oklahoma State,20,14,6,0.7,12.27,7.12,7,6,8,3,6,3,1515.0,1412.0,815.0,543.0,1196.0,0.454,133.0,391.0,0.34,296.0,423.0,0.7,227.0,788.0,269.0,151.0,92.0,311.0,340.0


In [37]:
predictions = reg3.predict(games)

In [38]:
predictions = pd.DataFrame(predictions, columns = ["score_visitor", "score_home"])

In [39]:
predictions.head()

,score_visitor,score_home
0,63.125500,77.756833
1,73.550167,65.465667
2,66.987000,94.336000
3,80.163333,68.168333
4,73.667500,61.309333


In [40]:
teams = teams.reset_index(drop=True)
predictions = predictions.reset_index(drop=True)
predictions.head(5)

,score_visitor,score_home
0,63.125500,77.756833
1,73.550167,65.465667
2,66.987000,94.336000
3,80.163333,68.168333
4,73.667500,61.309333


In [41]:
predictions = teams.join(predictions, how='inner')

In [42]:
predictions.columns

Index(['visitor_neutral', 'home_neutral', 'score_visitor', 'score_home'], dtype='object')

In [43]:
predictions["line"] = predictions["score_home"] - predictions["score_visitor"]

In [44]:
predictions

,visitor_neutral,home_neutral,score_visitor,score_home,line
0,North Alabama,Liberty,63.125500,77.756833,14.631333
1,Pacific,San Diego,73.550167,65.465667,-8.084500
2,Iowa State,Baylor,66.987000,94.336000,27.349000
3,Illinois,Michigan State,80.163333,68.168333,-11.995000
4,Oklahoma,Kansas State,73.667500,61.309333,-12.358167
5,St. John's (NY),Villanova,72.325000,80.480000,8.155000
6,Kansas,Texas,70.436667,76.926667,6.490000
7,Georgia Tech,Virginia Tech,64.716667,72.873333,8.156667
8,Akron,Ohio,75.164667,74.845667,-0.319000
9,Appalachian State,Georgia State,63.781690,73.440643,9.658952
